In [1]:
import qcodes as qc
station = qc.Station(config_file='config/station.yaml')

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : /Users/caijiaqi/.qcodes/logs/command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : /Users/caijiaqi/.qcodes/logs/251229-59997-qcodes.log


In [2]:
dac1 = station.load_mock_dac1()
dac2 = station.load_mock_dac2()

In [3]:
%load_ext instrmcp.extensions

2025-12-29 15:55:21,478 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 110 ¦ /Users/caijiaqi/GitHub/instrMCP/instrmcp/servers/qcodes/station_init.py:10: QCoDeSDeprecationWarning: The `qcodes.instrument.base` module is deprecated. Please consult the api documentation at https://microsoft.github.io/Qcodes/api/index.html for alternatives.
  from qcodes.instrument.base import Instrument

2025-12-29 15:55:21,963 ¦ qcodes.station ¦ WARNING ¦ station ¦ load_config_file ¦ 414 ¦ Could not load default config for Station: 
File /Users/caijiaqi/GitHub/MeasureIt\cfg\default.station.yaml not found. 
You can change the default config file in `qcodesrc.json`.
2025-12-29 15:55:21,985 ¦ qcodes.station ¦ WARNING ¦ station ¦ load_config_file ¦ 414 ¦ Could not load default config for Station: 
File /Users/caijiaqi/GitHub/MeasureIt\cfg\default.station.yaml not found. 
You can change the default config file in `qcodesrc.json`.
2025-12-29 15:55:21,985 ¦ qcodes.station ¦ WARNING ¦ station ¦ load_config_f


MeasureIt data directory: /Users/caijiaqi/GitHub/MeasureIt
To change: measureit.set_data_dir('/path')

✅ QCoDeS MCP extension loaded
🛡️  Default mode: safe
🔍 Auto-enabled: auto_correct_json, database, measureit
📋 Use %mcp_status to check server status
⚠️  Use %mcp_unsafe to switch to unsafe mode (if needed)
🚀 Use %mcp_start to start the server


In [3]:
%mcp_status

🛡️ MCP Server Status:
   Desired Mode: safe
   Server Running: ✅
   Host: 127.0.0.1:8123
   Task: ❌ Inactive
   Available: %mcp_close, %mcp_restart


In [6]:
%mcp_option add measureit database auto_correct_json

✅ Added: measureit
✅ Added: database
✅ Added: auto_correct_json
✅ Changes will take effect when server starts


In [4]:
%mcp_unsafe

⚠️  Mode set to unsafe
⚠️  UNSAFE MODE: execute_editing_cell tool will be available
⚠️  Server restart required for tool changes to take effect
   Use: %mcp_restart


In [8]:
%mcp_dangerous

⚠️⚠️⚠️  DANGEROUS MODE ENABLED  ⚠️⚠️⚠️
All consent dialogs will be automatically approved!
This mode bypasses all safety confirmations.
✅ Mode will take effect when server starts


In [9]:
%mcp_start

2025-11-30 15:50:58,931 ¦ instrmcp.servers.jupyter_qcodes.security.consent ¦ WARNING ¦ consent ¦ __init__ ¦ 61 ¦ ⚠️  CONSENT BYPASS MODE ENABLED - All operations auto-approved!
2025-11-30 15:50:58,937 ¦ instrmcp.servers.jupyter_qcodes.security.consent ¦ WARNING ¦ consent ¦ __init__ ¦ 61 ¦ ⚠️  CONSENT BYPASS MODE ENABLED - All operations auto-approved!


🚀 Starting MCP server...
🚀 QCoDeS MCP Server running on http://127.0.0.1:8123
🔑 Access token: YI1Og87_XIP8kZCsJ_EIk-9VP-czZGpDGFYqMwwAjZw
✅ MCP server started in ☠️ dangerous mode
⚠️⚠️⚠️  All consent dialogs auto-approved!


[11/30/25 15:50:58] INFO     Starting MCP server 'Jupyter QCoDeS MCP Server (Unsafe Mode)' with      ]8;id=935844;file:///Users/caijiaqi/miniforge3/envs/instrMCPdev/lib/python3.13/site-packages/fastmcp/server/server.py\server.py]8;;\:]8;id=27122;file:///Users/caijiaqi/miniforge3/envs/instrMCPdev/lib/python3.13/site-packages/fastmcp/server/server.py#2050\2050]8;;\
                             transport 'http' on http://127.0.0.1:8123/mcp                                         

INFO:     Started server process [40429]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 48] error while attempting to bind on address ('127.0.0.1', 8123): [errno 48] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


In [5]:
2222

2222

In [ ]:
y = 200

In [ ]:
print("Hello from new cell!")

print("Hello, testing output!")

In [2]:
raise ValueError("This is a test error!")

ValueError: This is a test error!

In [2]:
print("Testing output")
x = [1, 2, 3, 4, 5]
x

Testing output


[1, 2, 3, 4, 5]

In [4]:
result = 42 * 2
print(f"The answer is: {result}")
result

The answer is: 84


84

In [4]:
# Create a 120s 0D sweep following dac1.ch01.voltage
from measureit import Sweep0D

sweep_0d = Sweep0D(120, inter_delay=1, save_data=False, plot_data=True)
sweep_0d.follow_param(dac1.ch01.voltage)

# Start the sweep (non-blocking)
%gui qt
sweep_0d.start()

2025-12-29 15:57:43,479 | measureit.sweeps.Sweep0D | INFO | Done with the sweep, t=120.42734554130584 (s)
